In [ ]:
import logging
import json
import boto3
import pymysql
import csv
from datetime import datetime

logger = logging.get#Logger()
logger.setLevel(logging.INFO)

def lambda_handler(event, context):
    s3 = boto3.client('s3')
    
    try:
        bucket_name = event['Records'][0]['s3']['bucket']['name']
        object_key = event['Records'][0]['s3']['object']['key']
        #logger.info(f"Bucket: {bucket_name}, Key: {object_key}")
        
        # 파일 이름에서 이름 매핑을 찾는 코드 CSV 파일의 키를 정의
        name_mapping_key = 'code/code.csv'
        
    except KeyError as e:
        #logger.error(f"Invalid event format: {str(e)}")
        return {
            'statusCode': 400,
            'body': json.dumps(f"Invalid event format: {str(e)}")
        }
    except Exception as e:
        #logger.error(f"not found code.csv: {str(e)}")
        return {
            'statusCode': 500,
            'body': json.dumps(f"Internal server error: {str(e)}")
        }

    try:
        # 실시간 데이터 CSV 파일 읽기
        response = s3.get_object(Bucket=bucket_name, Key=object_key)
        lines = response['Body'].read().decode('utf-8-sig').splitlines()
        reader = csv.DictReader(lines)
        #logger.info(f"Successfully read CSV from S3")
        
        # 코드 CSV 파일 읽기 (이름 매핑)
        name_mapping_response = s3.get_object(Bucket=bucket_name, Key=name_mapping_key)
        name_mapping_lines = name_mapping_response['Body'].read().decode('utf-8-sig').splitlines()
        name_mapping_reader = csv.DictReader(name_mapping_lines)
        
        # 매핑 생성->딕셔너리 형태로
        name_mapping = {row['Code']: row['Name'] for row in name_mapping_reader}
        
    except Exception as e:
        #logger.error(f"Error getting object from S3: {str(e)}")
        return {
            'statusCode': 500,
            'body': json.dumps(f"Error getting object from S3: {str(e)}")
        }


    rds_host = 'antsdb.ctci2u4c45u3.ap-northeast-2.rds.amazonaws.com'
    username = 'ants'
    password = 'ants1234'
    db_name = 'datawarehouse'
    
    
    
    # CSV 컬럼 -> DB 컬럼 이름 매핑
    column_mapping = {
        'stock_code': 'stock_code',
        'name': 'name',
        'bstp_kor_isnm': 'sector',
        'rprs_mrkt_kor_name': 'market',
        'iscd_stat_cls_code':'status_code',
        'stck_prpr': 'current_price',
        'prdy_vrss' : 'UpDownRate',
        'prdy_vrss_sign' : 'PlusMinus',
        'UpDownPoint' : "UpDownPoint",
        'stck_oprc': 'opening_price',
        'stck_hgpr': 'high_price',
        'stck_lwpr': 'low_price',
        'price_time': 'price_time',
        'per' : 'per',
        'pbr' : 'pbr',
    }
    
    desired_columns = list(column_mapping.keys())

    
    try:
        conn = pymysql.connect(host=rds_host, user=username, passwd=password, db=db_name)
        cursor = conn.cursor()
        #logger.info("Successfully connected to RDS")
    except Exception as e:
        #logger.error(f"Error connecting to RDS: {str(e)}")
        return {
            'statusCode': 500,
            'body': json.dumps(f"Error connecting to RDS: {str(e)}")
        }

    try:
        for row in reader:
            #logger.info(f"Processing row: {row}")
            
            stock_code = row.get('stock_code', None)
            
            if not stock_code:
                #logger.error("Missing 'stock_code' in CSV data")
                raise ValueError("Missing 'stock_code' in CSV data")
            
            # 이름 매핑 추가
            name = name_mapping.get(stock_code, None)
            row['name'] = name
            up_down_rate = float(row.get('prdy_vrss', 0))
            current_price = float(row.get('stck_prpr', 0))
            
            # UpDownPoint 계산
            if current_price + up_down_rate != 0:  # 분모가 0이 되지 않도록
                row['UpDownPoint'] = up_down_rate / (current_price + up_down_rate)
            else:
                row['UpDownPoint'] = None
            
            # 'price_time' 변환: %y%m%d%H%M -> %Y-%m-%d %H:%M:%S
            if 'price_time' in row and row['price_time']:
                try:
                    row['price_time'] = datetime.strptime(row['price_time'], '%y%m%d%H%M').strftime('%Y-%m-%d %H:%M:%S')
                except ValueError as ve:
                    #logger.error(f"Invalid datetime format in 'price_time': {row['price_time']}")
                    raise ve
            
            # 누락된 컬럼 처리
            for col in desired_columns:
                if col not in row or row[col] is None:
                    row[col] = None  # 기본값 설정 (예: None)

            # 데이터 전처리: 원하는 컬럼만 필터링
            filtered_row = {column_mapping[col]: row[col] for col in desired_columns if col in row}
            #logger.info(f"Filtered row: {filtered_row}")
            
            # 테이블 확인
            cursor.execute(f"SHOW TABLES LIKE 'real_time'")
            result = cursor.fetchone()
            #logger.info(f"Table check result for real_time: {result}")
            
            if not result:
                #logger.info(f"Table real_time does not exist. Creating table.")
                
                # 각 컬럼에 대해 적절한 데이터 타입을 지정
                columns = {
                    'stock_code': 'VARCHAR(20)',
                    'name': 'VARCHAR(100)',
                    'sector': 'VARCHAR(100)',
                    'market': 'VARCHAR(10)',
                    'status_code':'VARCHAR(20)',
                    'current_price': 'int',
                    'UpDownRate' : "int",
                    'PlusMinus': "int",
                    "UpDownPoint":"DECIMAL(10, 2)",
                    'opening_price': 'int',
                    'high_price': 'int',
                    'low_price': 'int',
                    'price_time': 'DATETIME',
                    'per': 'DECIMAL(10, 2)',
                    "pbr": 'DECIMAL(10, 2)'
                }
                
                # SQL 생성
                create_table_sql = f"CREATE TABLE `real_time` ("
                create_table_sql += ", ".join([f"`{col}` {columns[col]}" for col in filtered_row.keys()])
                create_table_sql += ")"
                #logger.info(f"Executing SQL: {create_table_sql}")
                cursor.execute(create_table_sql)
            
            # 데이터를 테이블에 추가 (append)
            columns = ', '.join([f"`{col}`" for col in filtered_row.keys()])
            placeholders = ', '.join(['%s'] * len(filtered_row))
            sql = f"INSERT INTO `real_time` ({columns}) VALUES ({placeholders})"
            #logger.info(f"Executing SQL: {sql} with values {list(filtered_row.values())}")
            cursor.execute(sql, list(filtered_row.values()))
        
        conn.commit()
        #logger.info("Data inserted successfully")
    
    except Exception as e:
        #logger.error(f"Error inserting data: {str(e)}")
        conn.rollback()
        return {
            'statusCode': 500,
            'body': json.dumps(f"Error inserting data into RDS: {str(e)}")
        }
    finally:
        cursor.close()
        conn.close()
    return {
        'statusCode': 200,
        'body': json.dumps("Data inserted successfully")
    }
